# Processing Whole Exome Sequencing (WES/WXS) Data

In [2]:
!pwd

/home/fmbuga/ERR1831349


In [3]:
!ls

230722_P_fastq_2_vcf.ipynb


## Download data as fastq from NCBI SRA

In [6]:
!cat fasterq_dump_230722_slurm.sh

#!/bin/bash
#
#SBATCH --job-name=fasterq_dump_230722
#SBATCH --output=/home/fmbuga/tools/slurm_scripts/slurm_out_err/fasterq_dump_230722_%j.out
#SBATCH --error=/home/fmbuga/tools/slurm_scripts/slurm_out_err/fasterq_dump_230722_%j.err
#
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=28
#SBATCH --mem=0

datenow=$(date)
echo $datenow
echo ""
srun hostname
echo ""

start=$(date +%s)
echo "start time: $start"
echo ""
echo "hostname: $HOSTNAME"
echo ""


##################

eval "$(conda shell.bash hook)"
conda activate sra-tools                      ### CONDA ENVIRONMENT NAME
echo ""

work_dir=$(pwd)
echo "working directory: $work_dir"     ### WORKING DIRECTORY
echo ""

ncbi_sra_accession=$1                   ### NCBI SRA archive accession ID to download

# fasterq-dump
call=$(fasterq-dump $ncbi_sra_accession \
    --threads 28 \
    --progress \
    --details \
    --split-files)                                   ### split-files for paired-end data
    
echo $call
eval $call
echo "fasterq-dump

In [7]:
# submit fasterq-dump job to SLURM

!sbatch fasterq_dump_230722_slurm.sh \
    ERR1831349

!sleep 5
!squeue -u fmbuga
!sleep 10
!squeue -u fmbuga
!sleep 15
!squeue -u fmbuga
!sleep 30
!squeue -u fmbuga

Submitted batch job 645837
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            645836     nodes     bash   fmbuga  R    1:08:45      1 node18
            645837     nodes fasterq_   fmbuga  R       0:05      1 node35
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            645836     nodes     bash   fmbuga  R    1:08:55      1 node18
            645837     nodes fasterq_   fmbuga  R       0:15      1 node35
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            645836     nodes     bash   fmbuga  R    1:09:11      1 node18
            645837     nodes fasterq_   fmbuga  R       0:31      1 node35
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            645836     nodes     bash   fmbuga  R    1:09:41      1 node18
            645837     nodes fasterq_   fmbuga  R       1:01      1 node35


In [3]:
# create folder to store raw fastq and move downloaded files there
!mkdir ./00_raw_fastq
!mv *.fastq ./00_raw_fastq
!ls ./00_raw_fastq

ERR1831349_1.fastq  ERR1831349_2.fastq


## Check Quality of fastq with FastQC

In [6]:
!cat fastqc_allinFolder_230723_slurm.sh

#!/bin/bash
#
#SBATCH --job-name=fastqc_allInFolder_230723
#SBATCH --output=/home/fmbuga/tools/slurm_scripts/slurm_out_err/fastqc_allInFolder_230723_%j.out
#SBATCH --error=/home/fmbuga/tools/slurm_scripts/slurm_out_err/fastqc_allInFolder_230723_%j.err
#
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=2
#SBATCH --mem=0

datenow=$(date)
echo $datenow
echo ""
srun hostname
echo ""

start=$(date +%s)
echo "start time: $start"
echo ""
echo "hostname: $HOSTNAME"
echo ""


##################

eval "$(conda shell.bash hook)"
conda activate sra-tools                      ### CONDA ENVIRONMENT NAME
conda info
echo ""


call="fastqc --version"
echo $call
eval $call
echo ""

input_dir=$1             ### INPUT DIRECTORY
output_dir=$2            ### OUTPUT DIRECTORY 
echo "output directory: $output_dir"
echo ""

# fastqc

call=$(find $input_dir -name "*.fastq" | \
        xargs fastqc \
                --outdir $output_dir \
                --threads 2)                                   ### INPUT FASTQs


In [7]:
# submit fastqc job to SLURM
!sbatch fastqc_allinFolder_230723_slurm.sh \
    ./00_raw_fastq \
    ./00_raw_fastq

!sleep 5
!squeue -u fmbuga
!sleep 10
!squeue -u fmbuga
!sleep 15
!squeue -u fmbuga
!sleep 30
!squeue -u fmbuga

Submitted batch job 645885
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            645884     nodes     bash   fmbuga  R      33:05      1 node59
            645885     nodes fastqc_a   fmbuga  R       0:05      1 node60
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            645884     nodes     bash   fmbuga  R      33:15      1 node59
            645885     nodes fastqc_a   fmbuga  R       0:15      1 node60
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            645884     nodes     bash   fmbuga  R      33:30      1 node59
            645885     nodes fastqc_a   fmbuga  R       0:30      1 node60
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            645884     nodes     bash   fmbuga  R      34:01      1 node59
            645885     nodes fastqc_a   fmbuga  R       1:01      1 node60


In [11]:
# Import libraries for HTML visualization
from IPython.display import IFrame

In [12]:
IFrame(src='./00_raw_fastq/ERR1831349_1_fastqc.html', width=1500, height=2000)

In [13]:
IFrame(src='./00_raw_fastq/ERR1831349_2_fastqc.html', width=1500, height=2000)

- since no red flags in FastQC, proceed with mapping
- if mapping doesn't pass QC checks may return and see if some trimming helps

## map reads to human reference hg38 with bwa mem

In [16]:
!cat bwa_mem_hg38_samtools_fastq_to_bam_230723_slurm.sh

#!/bin/bash
#
#SBATCH --job-name=bwa_mem_hg38_samtools_fastq_to_bam_230723
#SBATCH --output=/home/fmbuga/tools/slurm_scripts/slurm_out_err/bwa_mem_hg38_samtools_fastq_to_bam_230723_%j.out
#SBATCH --error=/home/fmbuga/tools/slurm_scripts/slurm_out_err/bwa_mem_hg38_samtools_fastq_to_bam_230723_%j.err
#
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=28
##SBATCH --mem-per-cpu=4G

datenow=$(date)
echo $datenow
echo ""
srun hostname
echo ""

start=$(date +%s)
echo "start time: $start"
echo ""
echo "hostname: $HOSTNAME"
echo ""


##################

eval "$(conda shell.bash hook)"
conda activate gatk4                      ### CONDA ENVIRONMENT NAME
conda info
echo ""


call="bwa" # to get the version
echo "bwa version"
echo $call
eval $call
echo ""

call="samtools --version"
echo $call
eval $call
echo ""

fastq1=$1                             ### INPUT FASTQ1
fastq2=$2                             ### INPUT FASTQ2
output_dir=$3                         ### OUTPUT DIRECTORY 

echo "output directory:

In [15]:
# create directory to output bam files to
!mkdir ./01_raw_bam

In [1]:
# submit mapping job to SLURM HPC
!sbatch bwa_mem_hg38_samtools_fastq_to_bam_230723_slurm.sh \
    ./00_raw_fastq/ERR1831349_1.fastq \
    ./00_raw_fastq/ERR1831349_2.fastq \
    ./01_raw_bam/

!sleep 5
!squeue -u fmbuga
!sleep 10
!squeue -u fmbuga
!sleep 15
!squeue -u fmbuga
!sleep 30
!squeue -u fmbuga

Submitted batch job 645893
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            645891     nodes     bash   fmbuga  R      28:21      1 node58
            645893     nodes bwa_mem_   fmbuga  R       0:05      1 node05
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            645891     nodes     bash   fmbuga  R      28:31      1 node58
            645893     nodes bwa_mem_   fmbuga  R       0:15      1 node05
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            645891     nodes     bash   fmbuga  R      28:46      1 node58
            645893     nodes bwa_mem_   fmbuga  R       0:30      1 node05
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            645891     nodes     bash   fmbuga  R      29:16      1 node58
            645893     nodes bwa_mem_   fmbuga  R       1:00      1 node05


In [2]:
# QC check output bam file
!samtools flagstat \
    -@ 28 \
    ./01_raw_bam/ERR1831349.bam

93603695 + 0 in total (QC-passed reads + QC-failed reads)
93556700 + 0 primary
0 + 0 secondary
46995 + 0 supplementary
0 + 0 duplicates
0 + 0 primary duplicates
93403023 + 0 mapped (99.79% : N/A)
93356028 + 0 primary mapped (99.79% : N/A)
93556700 + 0 paired in sequencing
46778350 + 0 read1
46778350 + 0 read2
92600884 + 0 properly paired (98.98% : N/A)
93279060 + 0 with itself and mate mapped
76968 + 0 singletons (0.08% : N/A)
593850 + 0 with mate mapped to a different chr
517790 + 0 with mate mapped to a different chr (mapQ>=5)


- based on the high mapping rate (~99%) no need to go back and trim fastq reads
- proceed with marking duplicates

## Mark Duplicates

In [9]:
!cat gatk_markDuplicates_230724_slurm.sh

#!/bin/bash
#
#SBATCH --job-name=gatk_markDuplicates_221003
#SBATCH --output=/home/fmbuga/tools/slurm_scripts/slurm_out_err/gatk_markDuplicates_221003_%j.out
#SBATCH --error=/home/fmbuga/tools/slurm_scripts/slurm_out_err/gatk_markDuplicates_221003_%j.err
#
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=28
#SBATCH --mem=0

datenow=$(date)
echo $datenow
echo ""
srun hostname
echo ""

start=$(date +%s)
echo "start time: $start"
echo ""
echo "hostname: $HOSTNAME"
echo ""


##################

eval "$(conda shell.bash hook)"
conda activate gatk4                      ### CONDA ENVIRONMENT NAME
conda info
echo ""


call="gatk MarkDuplicates --version" # to get the version
echo $call
eval $call
echo ""

call="gatk MarkDuplicatesSpark --version" # to get the version
echo $call
eval $call
echo ""

call="gatk MarkDuplicates --help" # help
echo $call
eval $call
echo ""

call="gatk MarkDuplicatesSpark --help" # help
echo $call
eval $call
echo ""


input_bam=$1                            ### INPUT BAM
o

In [10]:
# submit mark duplicates job to SLURM HPC
!sbatch gatk_markDuplicates_230724_slurm.sh \
    ./01_raw_bam/ERR1831349.bam \
    ./02_bam_markdup/

!sleep 5
!squeue -u fmbuga
!sleep 10
!squeue -u fmbuga
!sleep 15
!squeue -u fmbuga
!sleep 30
!squeue -u fmbuga

Submitted batch job 646016
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            646009     nodes     bash   fmbuga  R    1:31:36      1 node58
            646016     nodes gatk_mar   fmbuga  R       0:05      1 node25
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            646009     nodes     bash   fmbuga  R    1:31:46      1 node58
            646016     nodes gatk_mar   fmbuga  R       0:15      1 node25
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            646009     nodes     bash   fmbuga  R    1:32:02      1 node58
            646016     nodes gatk_mar   fmbuga  R       0:31      1 node25
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            646009     nodes     bash   fmbuga  R    1:32:32      1 node58
            646016     nodes gatk_mar   fmbuga  R       1:01      1 node25


In [11]:
!samtools flagstat \
    -@ 28 \
    ./02_bam_markdup/ERR1831349_markdup.bam

93603695 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
46995 + 0 supplementary
34258723 + 0 duplicates
93403023 + 0 mapped (99.79% : N/A)
93556700 + 0 paired in sequencing
46778350 + 0 read1
46778350 + 0 read2
92600884 + 0 properly paired (98.98% : N/A)
93279060 + 0 with itself and mate mapped
76968 + 0 singletons (0.08% : N/A)
593850 + 0 with mate mapped to a different chr
517790 + 0 with mate mapped to a different chr (mapQ>=5)


## Variant Calling

In [12]:
# make folder to store vcf
!mkdir 03_vcf_raw

In [4]:
!cat bcftools_mpileup_call_P099_230724_slurm.sh

#!/bin/bash
#
#SBATCH --job-name=bcftools_mpileup_call_ONT_P099_221129
#SBATCH --output=/home/fmbuga/tools/slurm_scripts/slurm_out_err/bcftools_mpileup_call_ONT_P099_221129_%j.out
#SBATCH --error=/home/fmbuga/tools/slurm_scripts/slurm_out_err/bcftools_mpileup_call_ONT_P099_221129_%j.err
#
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=0


# "...multithreading ...is currently used only for the compression of the output stream, only when --output-type is b or z"
# https://samtools.github.io/bcftools/bcftools.html
# "Use the -Ou option when piping between bcftools subcommands to speed up performance by removing unnecessary compression/decompression and VCF←→BCF conversion"
# for ONT https://github.com/samtools/bcftools/issues/1666

# "For ONT it may be beneficial to also run bcftools call with a higher -P, eg -P0.01 or -P 0.1"


datenow=$(date)
echo $datenow
srun hostname
echo ""

start=$(date +%s)
echo "start time: $start"
echo "hostname: $HOSTNAME"
echo ""


################

In [5]:
# submit bcftools mpileup call variants job to SLURM HPC
!sbatch bcftools_mpileup_call_P099_230724_slurm.sh \
    ./02_bam_markdup/ERR1831349_markdup.bam \
    ./03_vcf_raw/

!sleep 5
!squeue -u fmbuga
!sleep 10
!squeue -u fmbuga
!sleep 15
!squeue -u fmbuga
!sleep 30
!squeue -u fmbuga

Submitted batch job 646019
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            646017     nodes     bash   fmbuga  R      17:31      1 node58
            646019     nodes bcftools   fmbuga  R       0:04      1 node25
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            646017     nodes     bash   fmbuga  R      17:42      1 node58
            646019     nodes bcftools   fmbuga  R       0:15      1 node25
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            646017     nodes     bash   fmbuga  R      17:57      1 node58
            646019     nodes bcftools   fmbuga  R       0:30      1 node25
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            646017     nodes     bash   fmbuga  R      18:27      1 node58
            646019     nodes bcftools   fmbuga  R       1:00      1 node25


In [2]:
!bcftools stats \
    --threads 28 \
    03_vcf_raw/ERR1831349_markdup_bcftools_P_099.vcf | grep ^SN

SN	0	number of samples:	1
SN	0	number of records:	2827413
SN	0	number of no-ALTs:	0
SN	0	number of SNPs:	2767029
SN	0	number of MNPs:	0
SN	0	number of indels:	60384
SN	0	number of others:	0
SN	0	number of multiallelic sites:	17138
SN	0	number of multiallelic SNP sites:	1595


In [9]:
# filter to keep SNVs only
!cat bcftools_filter_snps_only_230725_slurm.sh

#!/bin/bash
#
#SBATCH --job-name=bcftools_filter_snps_only_230725
#SBATCH --output=/home/fmbuga/tools/slurm_scripts/slurm_out_err/bcftools_filter_snps_only_230725_%j.out
#SBATCH --error=/home/fmbuga/tools/slurm_scripts/slurm_out_err/bcftools_filter_snps_only_230725_%j.err
#
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=28
#SBATCH --mem=0


datenow=$(date)
echo $datenow
srun hostname
echo ""

start=$(date +%s)
echo "start time: $start"
echo "hostname: $HOSTNAME"
echo ""


##################

eval "$(conda shell.bash hook)"
conda activate minimap2_224                      ### CONDA ENVIRONMENT NAME
conda info
echo ""


call="bcftools --version"
echo $call
eval $call
echo ""

input_vcf=$1
output_dir=$2

filename=$(basename $1 .vcf)


call=$(bcftools filter \
    -i 'TYPE="snp"' \
    $input_vcf \
    --threads 28 \
    -o "$output_dir$filename"_snps_only.vcf)
    
    
echo $call
eval $call
echo ""

echo "bcftools filter keep snps only COMPLETE."
echo ""
    
##################


echo ""
end

In [11]:
# create folder to store snps_only vcf
!mkdir ./04_vcf_snps_only/

In [12]:
# submit snps filter job to SLURM HPC
!sbatch bcftools_filter_snps_only_230725_slurm.sh \
    ./03_vcf_raw/ERR1831349_markdup_bcftools_P_099.vcf \
    ./04_vcf_snps_only/

!sleep 5
!squeue -u fmbuga
!sleep 10
!squeue -u fmbuga
!sleep 15
!squeue -u fmbuga
!sleep 30
!squeue -u fmbuga

Submitted batch job 646161
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            646156     nodes     bash   fmbuga  R      41:12      1 node25
            646161     nodes bcftools   fmbuga  R       0:04      1 node58
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            646156     nodes     bash   fmbuga  R      41:23      1 node25
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            646156     nodes     bash   fmbuga  R      41:38      1 node25
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            646156     nodes     bash   fmbuga  R      42:08      1 node25


In [13]:
!bcftools stats \
    --threads 28 \
    04_vcf_snps_only/ERR1831349_markdup_bcftools_P_099_snps_only.vcf | grep ^SN

SN	0	number of samples:	1
SN	0	number of records:	2767029
SN	0	number of no-ALTs:	0
SN	0	number of SNPs:	2767029
SN	0	number of MNPs:	0
SN	0	number of indels:	0
SN	0	number of others:	0
SN	0	number of multiallelic sites:	1595
SN	0	number of multiallelic SNP sites:	1595


## Only Keep Calls that Intersect with Exome Target Regions and GIAB Truth Set

In [14]:
%%time
# intersect vcf with GIAB truth set high-confidence regions BED

!mkdir ./05_vcf_snps_only_GIAB_isec/

!bedtools intersect \
    -a 04_vcf_snps_only/ERR1831349_markdup_bcftools_P_099_snps_only.vcf \
    -b ~/tools/GIAB/GRCh38/HG001_NA12878/NA12878_HG001_GRCh38_GIAB.bed \
    -header > 05_vcf_snps_only_GIAB_isec/ERR1831349_markdup_bcftools_P_099_snps_only_GIAB_isec.vcf

CPU times: user 72.2 ms, sys: 25.1 ms, total: 97.3 ms
Wall time: 4.92 s


In [15]:
!bcftools stats \
    --threads 28 \
    05_vcf_snps_only_GIAB_isec/ERR1831349_markdup_bcftools_P_099_snps_only_GIAB_isec.vcf | grep ^SN

SN	0	number of samples:	1
SN	0	number of records:	2099550
SN	0	number of no-ALTs:	0
SN	0	number of SNPs:	2099550
SN	0	number of MNPs:	0
SN	0	number of indels:	0
SN	0	number of others:	0
SN	0	number of multiallelic sites:	886
SN	0	number of multiallelic SNP sites:	886


In [16]:
%%time
# intersect vcf with Agilent_SureSelect_Human_All_Exon_V4 exome target regions BED

!mkdir ./06_vcf_snps_only_GIAB_WXS_isec/

!bedtools intersect \
    -a 05_vcf_snps_only_GIAB_isec/ERR1831349_markdup_bcftools_P_099_snps_only_GIAB_isec.vcf \
    -b ~/tools/Agilent/SureSelect/SureSelect_Human_All_Exon_V4/Agilent_S04380110_hs_hg19/S04380110_Covered_hg19toHg38.bed \
    -header > 06_vcf_snps_only_GIAB_WXS_isec/ERR1831349_markdup_bcftools_P_099_snps_only_GIAB_WXS_isec.vcf

CPU times: user 39.6 ms, sys: 22.8 ms, total: 62.4 ms
Wall time: 2.74 s


In [17]:
!bcftools stats \
    --threads 28 \
    06_vcf_snps_only_GIAB_WXS_isec/ERR1831349_markdup_bcftools_P_099_snps_only_GIAB_WXS_isec.vcf | grep ^SN

SN	0	number of samples:	1
SN	0	number of records:	34676
SN	0	number of no-ALTs:	0
SN	0	number of SNPs:	34676
SN	0	number of MNPs:	0
SN	0	number of indels:	0
SN	0	number of others:	0
SN	0	number of multiallelic sites:	14
SN	0	number of multiallelic SNP sites:	14


In [18]:
# intersect GIAB high confidence vcf with Agilent_SureSelect_Human_All_Exon_V4 exome target regions BED to compare to ERR1831349 vcf

!bedtools intersect \
    -a ~/tools/GIAB/GRCh38/HG001_NA12878/NA12878_HG001_GRCh38_GIAB.vcf.gz \
    -b ~/tools/Agilent/SureSelect/SureSelect_Human_All_Exon_V4/Agilent_S04380110_hs_hg19/S04380110_Covered_hg19toHg38.bed \
    -header > 06_vcf_snps_only_GIAB_WXS_isec/NA12878_GIAB_WXS_Regions_hg19toHg38.vcf

In [19]:
!bcftools stats \
    --threads 28 \
    06_vcf_snps_only_GIAB_WXS_isec/NA12878_GIAB_WXS_Regions_hg19toHg38.vcf | grep ^SN

SN	0	number of samples:	1
SN	0	number of records:	35415
SN	0	number of no-ALTs:	0
SN	0	number of SNPs:	32811
SN	0	number of MNPs:	0
SN	0	number of indels:	2607
SN	0	number of others:	2
SN	0	number of multiallelic sites:	65
SN	0	number of multiallelic SNP sites:	10


In [20]:
# keep GIAB SNPs only

!bcftools filter \
    -i 'TYPE="snp"' \
    06_vcf_snps_only_GIAB_WXS_isec/NA12878_GIAB_WXS_Regions_hg19toHg38.vcf \
    --threads 28 \
    -o 06_vcf_snps_only_GIAB_WXS_isec/NA12878_GIAB_WXS_snps_only.vcf

In [21]:
!bcftools stats \
    --threads 28 \
    06_vcf_snps_only_GIAB_WXS_isec/NA12878_GIAB_WXS_snps_only.vcf | grep ^SN

SN	0	number of samples:	1
SN	0	number of records:	32806
SN	0	number of no-ALTs:	0
SN	0	number of SNPs:	32806
SN	0	number of MNPs:	0
SN	0	number of indels:	0
SN	0	number of others:	0
SN	0	number of multiallelic sites:	10
SN	0	number of multiallelic SNP sites:	10


```
NEXT STEP: use machine learning methods to filter the vcf
```